# Dependencies

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import networkx as nx
import netwulf as wulf
import datetime as dt
import shifterator as sh
import nltk
import re
import random
import json
import community

from wordcloud import WordCloud
from networkx.readwrite import json_graph
from collections.abc import Iterable
from collections import Counter
from scipy import stats

from data import get_master_dict

# Data preprocessing

A document in the corpus is 'General course objectives' + 'Learning objectives' + 'Content'.

## Load data

In [2]:
data = get_master_dict()

In [8]:
courses = pd.DataFrame.from_dict(data, orient='index')
courses

,description,language,ects,type,location,evaluation,prerequisites,responsible,co-responsible,department,grades,reviews,name,passpercent,avg,pp,avgp,qualityscore,workload,lazyscore
10380,General course objectives\r\nModern optics and...,English,10,MScOffered as a single course,Campus Lyngby,"7 step scale , external examiner","34020.10102 , Photonics and quantum\r\n mech...","\r\nAlexander Huck , Lyngby Campus, Building 3...","\r\nJesper Mørk , Lyngby Campus, Building 345A...",10 Department of\r\nPhysics,"{'-3': '0', '00': '0', '02': '1', '4': '2', '7...","[{'participants': 10, 'timestamp': 'E21', '1.1...",Kvanteoptik,81.0,9.0,67.2,70.8,8.6,40.2,33.7
47335,"General course objectives\r\nIn this course, t...",English,5,MSc,This is a e-learning course with online videos...,"7 step scale , internal examiner",Basic concepts from physical chemistry and/or ...,"\r\nIvano Eligio Castelli , Lyngby Campus, Bui...","\r\nPiotr de Silva , Lyngby Campus, Building 3...",47\r\nDepartment of Energy Conversion and Storage,"{'-3': '0', '00': '0', '02': '0', '4': '0', '7...","[{'participants': 7, 'timestamp': 'E21', '1.1'...",Computermodellering af materialer til\r\nenerg...,85.0,10.4,74.1,85.4,57.3,53.3,52.9
27016,General course objectives\r\nTo be immersed in...,English,5,BSc,Campus Lyngby,"pass / not pass , internal examiner",NaN,"\r\nChristopher Workman , Lyngby Campus, Build...","\r\nArto Heiskanen , Lyngby Campus, Building 4...",27\r\nDepartment of Biotechnology and Biomedicine,{},"[{'participants': 97, 'timestamp': ' 22', '1.1...",Design-Build 1: Devices for measuring cell growth,87.0,NaN,77.6,NaN,44.2,59.1,61.1
62t37,"General course objectives\r\nIn this course, t...",Danish,5,DeltidsdiplomOffered as a single course,Campus Ballerup,"7 step scale , external examiner",Admission requirements: - That the applicant h...,"\r\nHenrik Tange , Ballerup Campus, Building B...","\r\nLynn Eriksen , Ballerup Campus, Building B...",\r\n62 Department of Engineering Technology an...,"{'-3': '0', '00': '0', '02': '0', '4': '0', '7...","[{'participants': 4, 'timestamp': 'F18', '1': ...",Cyber defence and legal aspects,100.0,9.9,100.0,80.2,NaN,NaN,NaN
25616,General course objectives\r\nTo familiarize th...,English,5,"Ph.D., Professionally focussed courseOffered a...",The two first weeks take place at North Sea Sc...,"pass / not pass , internal examiner",NaN,"\r\nLudvig Ahm Krag , Hirtshals , Building Hov...",NaN,25\r\nNational Institute of Aquatic Resources,{},NaN,Fangstteknologi,100.0,NaN,100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25322,General course objectives\r\nTo provide studen...,English,5,MScOffered as a single course,DTU Aqua in Hirtshals,"pass / not pass , internal examiner","25316 , or similar","\r\nPer Bovbjerg Pedersen , Hirtshals , Buildi...",NaN,25\r\nNational Institute of Aquatic Resources,{},"[{'participants': 5, 'timestamp': ' 21', '1.1'...",Advanced course in recirculating aquaculture s...,100.0,NaN,100.0,NaN,0.0,74.6,89.9
62365,General course objectives\r\nThe student shall...,English,5,BEngOffered as a single course,Campus Ballerup,"7 step scale , external examiner","62325/62328 , It is recommented that you have\...","\r\nJohn Køhler Carlsson , Ballerup Campus, Bu...",NaN,\r\n62 Department of Engineering Technology an...,"{'-3': '0', '00': '2', '02': '0', '4': '4', '7...","[{'participants': 4, 'timestamp': 'F21', '1.1'...",Byggegruber,80.0,4.4,65.5,22.9,NaN,NaN,NaN
23272,General course objectives\r\nThe course provid...,English,10,MScOffered as a single course,Campus Lyngby,"7 step scale , internal examiner",NaN,"\r\nHåkan Vigre , Lyngby Campus, Building 204 ...",NaN,23 National Food\r\nInstitute,"{'-3': '1', '00': '0', '02': '1', '4': '2', '7...","[{'participants': 15, 'timestamp': 'E21', '1.1...",Risikoanalyse i fødevaresikkerhed,88.0,7.6,79.3,56.2,2.7,18.2,24.1
62p37,General course objectives\r\nThe purpose of th...,Danish,5,DeltidsdiplomOffered as a single course,Copenhagen

## Tokenizer

In [4]:
# Stop words are removed from the tokenized text. We get the stop words here.
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words = [re.sub('[^a-z_-]', '', stop_word) for stop_word in stop_words]

def tokenize(text: str) -> list[str]:
    """Tokenizer function. 
    The following is removed from the tokenized text:
    Symbols, stop words.
    
    Finally, only unique words are included in the tokens (ie. no repeats)"""
    text = text.lower()
    text = re.sub('[^a-z]', ' ', text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
   
    return tokens

In [5]:
# The tokenized text is saved in a new column
courses['tokens'] = courses.apply(lambda row: tokenize(row['text']), axis=1)
courses.head()

,text,tokens
01005,The course content is the mathematical basis f...,"[course, content, mathematical, basis, broad, ..."
23932,"The students should, based on the cell, acquir...","[students, based, cell, acquire, basic, unders..."
10031,The overall goal is to give Physics and Nanote...,"[overall, goal, give, physics, nanotechnology,..."


In [8]:
# Create a corpus wich is a dictionary of the tokenized texts
corpus = courses['tokens'].to_dict()

# TF-IDF analysis

In [6]:
def term_frequency(document: list) -> tuple[np.array, np.array]:
    """This function calculates the term frequency of a document.
    The term frequency is normalized by the total number of words in the document.
    Returns a tuple of arrays. The first array is the terms and the second array is
    the term frequency in the same order.
    
    Arguments:
        document (np.array): array of tokens.
    """
    fdist = nltk.FreqDist(document)
    n_words = fdist.N()
    terms = np.array(list(fdist.keys()))
    tf = np.array(list(fdist.values())) / n_words
    
    return tf, terms


def inverse_document_frequency(documents: dict[list], corpus=None) -> dict[float]:
    """This function computes the inverse document frequency (idf) given
    a list of documents and a set of terms.
    """
    if corpus is None:
        corpus = documents.keys()
    
    n_documents = len(corpus)
    terms, counts = np.unique(
        np.concatenate([list(set(documents[d])) for d in corpus]),
        return_counts=True
    )
    
    idf = np.log(n_documents / (counts))
    idf_lookup = {term: w for term, w in zip(terms, idf)}
    
    return idf_lookup

def tf_idf(document: list, idf_lookup: dict[float]) -> tuple[np.array, np.array]:
    """Computes the tf-idf of a document, given a document and a lookup table for idf."""
    tf, terms = term_frequency(document)
    idf = np.array([idf_lookup[term] for term in terms])
    
    return tf * idf, terms

In [11]:
# Top 5 words, TF
print('Top 5 terms sorted by term frequency (TF):')

for course, document in corpus.items():
    tf, terms = term_frequency(document)
    sorted_indices = np.argsort(tf)[::-1]
    stock_top_5 = terms[sorted_indices[:5]]
    print(f'{course}:')
    print(stock_top_5)

Top 5 terms sorted by term frequency (TF):
01005:
['use' 'linear' 'equations' 'functions' 'applications']
23932:
['biological' 'cell' 'describe' 'structure' 'students']
10031:
['physics' 'sections' 'well' 'technology' 'within']


In [12]:
idf_lookup = inverse_document_frequency(corpus)

In [14]:
# Top 10 words, TF
print('Top 10 terms sorted by term frequency (TF):')

topx = 10
for course, document in corpus.items():
    tf, terms = term_frequency(document)
    sorted_indices = np.argsort(tf)[::-1]
    stock_top_10 = terms[sorted_indices[:topx]]
    # print(tf[sorted_indices[:topx]])
    print(f'{course}:')
    print(stock_top_10)
    
print('\n')

# Top 10 words, TF-IDF
print('Top 10 terms sorted by TF-IDF:')

for course, document in corpus.items():
    tfidf, terms = tf_idf(document, idf_lookup)
    sorted_indices = np.argsort(tfidf)[::-1]
    stock_top_10 = terms[sorted_indices[:topx]]
    # print(tfidf[sorted_indices[:topx]])
    print(f'{course}:')
    print(stock_top_10)

Top 10 terms sorted by term frequency (TF):
01005:
['use' 'linear' 'equations' 'functions' 'applications' 'elementary'
 'mathematical' 'systems' 'complex' 'differential']
23932:
['biological' 'cell' 'describe' 'structure' 'students' 'basic' 'processes'
 'explain' 'regulation' 'mechanisms']
10031:
['physics' 'sections' 'well' 'technology' 'within' 'subjects' 'laboratory'
 'work' 'data' 'engineering']


Top 10 terms sorted by TF-IDF:
01005:
['linear' 'equations' 'elementary' 'applications' 'differential'
 'mathematical' 'systems' 'complex' 'mathematics' 'vector']
23932:
['biological' 'cell' 'describe' 'basic' 'processes' 'regulation'
 'mechanisms' 'living' 'energy' 'pro']
10031:
['physics' 'sections' 'subjects' 'data' 'group' 'ultrafast' 'optical'
 'pulses' 'hydrodynamics' 'lab']
